# Visualization Code for OddLot

This Jupyter Notebook will hold all of the code for deriving the data in Charon SSG's slide deck.


## Base Code
This base code will be needed for all of the visualizations. So I'm putting it right at the start. This includes things such as importing needed modules and reading in and cleaning the data.

Estimated return is simply shares buying back/outstanding shares, discretized by the increments


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime

# Constants
####
# TODO: get montly treasury rate and average that across year
RISK_FREE_RATE = .0469 # MEAN 20 YEAR TREASURY RATE
####

odd_lot_data = pd.read_csv('./data/cleaned-data/url_data.csv')

# Remove any data rows where there have been errors or duplicates
odd_lot_data.dropna(subset=['exp date', 'trading px'], inplace=True)
odd_lot_data.drop_duplicates(subset=['cik', 'date'], inplace = True)

# Add average price
odd_lot_data['avg'] = (odd_lot_data['high'] - odd_lot_data['low']) / 2 + odd_lot_data['low']

# Make the date columns into actual python datetimes
odd_lot_data['date'] = pd.to_datetime(odd_lot_data['date'])
odd_lot_data['exp date'] = pd.to_datetime(odd_lot_data['exp date'])
odd_lot_data['px date'] = pd.to_datetime(odd_lot_data['px date'])
odd_lot_data['pay date'] = pd.to_datetime(odd_lot_data['exp date']).apply(lambda date: date +  datetime.timedelta(days=6))
# odd_lot_data['year'] = pd.DatetimeIndex(odd_lot_data['date']).year

# Only look at data inside of date range
start_date = pd.to_datetime('20000101', format='%Y%m%d', errors='ignore')
end_date = pd.to_datetime('20181231', format='%Y%m%d', errors='ignore')
odd_lot_data = odd_lot_data[(odd_lot_data['date'] > start_date) & (odd_lot_data['pay date'] < end_date)]
# print(odd_lot_data)

# Cast price cap and share cap as ints
odd_lot_data['price cap'] = odd_lot_data['price cap'].astype(float, errors='ignore') 
odd_lot_data['share cap'] = odd_lot_data['share cap'].astype(float, errors='ignore')
print(odd_lot_data)


odd_lot_data.sort_values('date', inplace=True)
odd_lot_data.reset_index(drop=True, inplace=True)

# Initialize fonts
georgia_font = {'fontname':'Georgia'}

              cik       date  \
0            1750 2015-04-27   
1            2135 2006-02-09   
2            2491 2011-04-08   
3            3370 2007-11-21   
4            3545 2018-09-05   
5            5117 2005-09-07   
6           12978 2005-03-30   
7           14693 2003-02-04   
8           14846 2010-09-22   
9           78749 2007-08-21   
10          78890 2006-03-09   
11         315858 2018-03-20   
13         318154 2011-11-08   
14         350698 2006-03-10   
15         701288 2001-11-26   
17         701288 2003-03-18   
18         703351 2006-08-29   
19         716459 2000-09-29   
20         722104 2006-08-14   
21         723209 2006-11-20   
22         723209 2007-08-15   
23         733269 2006-08-07   
24         733269 2018-11-13   
25         760498 2007-08-03   
26         782842 2011-09-14   
27         812482 2008-05-19   
28         824803 2003-04-23   
30         830656 2004-12-27   
32         832488 2015-11-02   
33         835540 2008-10-20   
..      

## Flag Outliers

If an offer has crazy high return, I've flagged it so that we can take a look at it and make sure the values are correct.

In [2]:
# def calculate_sell_price(row):
#     if np.isnan(row['low']):
#         return row['tender px']
#     else:
#         return row['low']

# sell_prices = odd_lot_data.apply(lambda row: calculate_sell_price(row), axis=1)
# odd_lot_data['buy_low_px_spread'] = sell_prices - odd_lot_data['trading px']
# odd_lot_data['low_norm_return'] = odd_lot_data['buy_low_px_spread'] / odd_lot_data['trading px']

# highest_returns = odd_lot_data.sort_values(by='low_norm_return').tail(15)
# print(highest_returns)

## Simulation Variables
This section will handle any variables that we want to test, so that we can see what their effect is on returns.

In [3]:
### SET SIMULATOR VARIABLES
# Decide whether we are using a one account limit per company
one_account_limit = False
# tender_price_appreciation - how close doed a tender price gets to the low price
tender_price_appreciation = 0 # (increments: [0, .25, .5, .75, 1])
# Failure rates - what percent of tender offers do I have to sell back at market px
failure_rate = 0 # (increments: [0, .05, .1, .25, .5, .75])
failure_penalty_rate = 0 # (increments: [.01, .02, .05, .1])
# The percentage of the share cap we can take at max
share_cap_modifier = .25 # (increments: [.1, .25, .5, .75])
# Fee rates - The fee rate for making a transaction
fee_rate = .1 # (increments: [0, .01, .02, .05, .1])

def get_appreciated_price(row):
    result = np.NaN
    if not np.isnan(row['low']):
        if row['low'] > row['trading px']:
            result = round(row['trading px'] + tender_price_appreciation * (row['low'] - row['trading px']), 2)
        else:
            result = row['trading px']
    else:
        if row['tender px'] > row['trading px']:
            result = round(row['trading px'] + tender_price_appreciation * (row['tender px'] - row['trading px']), 2)
        else:
            result = row['trading px']
    return result
#     print(row['low'])
#     print(row['tender px'])
#     print(row['trading px'])
#     print(result)
#     print('~~~~~~')

def get_our_share_cap(row, outlook='avg'):
    # Get sell price
    sell_price = row[outlook]
    if np.isnan(sell_price) or sell_price is None:
        sell_price = row['tender px']
        
    # If there is a share cap and price cap, pick the most limiting of the two given the outlook, as our share cap
    if row['share cap'] is not None and row['price cap'] is not None \
        and not np.isnan(row['share cap']) and not np.isnan(row['price cap']):
        price_share_cap = np.floor(float(row['price cap']) / float(sell_price))

        # Pick the most limiting of the two
        if row['share cap'] < price_share_cap:
            return np.double(row['share cap'] * share_cap_modifier)
        else:
            return price_share_cap * share_cap_modifier
        
    # If there is a no price cap, but there is a share cap, use that as our share cap
    elif row['share cap'] is not None and not np.isnan(row['share cap']):
        return row['share cap'] * share_cap_modifier
    
    # If there is a no share cap, but there is a share cap, use that as our share cap
    elif row['price cap'] is not None and not np.isnan(row['price cap']):
        price_share_cap = np.floor(row['price cap'] / sell_price)
        return price_share_cap * share_cap_modifier
    
    else:
        raise Exception("Must have a value for either the 'share cap' or 'price cap' columns")
        
if tender_price_appreciation > 0:
    odd_lot_data['trading px'] = odd_lot_data.apply(get_appreciated_price, axis=1)
odd_lot_data['our_share_cap'] = odd_lot_data.apply(get_our_share_cap, outlook='avg', axis=1)
odd_lot_data['fee_cost_per_share'] = fee_rate * odd_lot_data['trading px']
print(odd_lot_data)

              cik       date  \
0    "0001032208" 2000-01-26   
1    "0000096879" 2000-01-26   
2    "0000787250" 2000-02-04   
3    "0000033488" 2000-02-24   
4    "0000046080" 2000-02-28   
5    "0001021575" 2000-03-01   
6    "0000912766" 2000-03-21   
7    "0000785815" 2000-03-21   
8    "0000909954" 2000-04-17   
9    "0001036615" 2000-04-24   
10   "0000763744" 2000-05-01   
11   "0001059131" 2000-06-05   
12        1019439 2000-07-31   
13   "0000919568" 2000-08-01   
14   "0000912766" 2000-08-10   
15   "0000706698" 2000-08-18   
16   "0000776867" 2000-08-21   
17   "0000032020" 2000-09-08   
18   "0000083402" 2000-09-26   
19   "0001012771" 2000-09-26   
20   "0000716459" 2000-09-29   
21         716459 2000-09-29   
22   "0000769520" 2000-10-23   
23   "0000318189" 2000-10-27   
24   "0001006265" 2000-10-31   
25   "0000930420" 2000-11-06   
26   "0000108079" 2000-11-09   
27   "0000813298" 2000-11-15   
28   "0000216039" 2000-12-15   
29   "0000785024" 2000-12-15   
..      

## The Naive Strategy
Buy stocks of every tender lot offer, simple. Only do this if there is a one account limit. Multiple accounts will just use our attempted best strategy.

In [4]:
if one_account_limit:
    odd_lot_data['naive'] = 'y'
    print(odd_lot_data['naive'].value_counts())

## The Conservative Strategy
Only buy if the low price is above the initial buy price, ergo you can't ever lose money. Only do this if there is a one account limit. Multiple accounts will just use our attempted best strategy.

In [5]:
def conservative_strategy(row):
    if row['trading px'] <= row['low']:
        return 'y'
    elif row['trading px'] <= row['tender px']:
        return 'y'
    else:
        return 'n'
    
if one_account_limit:
    odd_lot_data['conservative'] = odd_lot_data.apply(lambda row: conservative_strategy(row), axis=1)
    print(odd_lot_data['conservative'].value_counts())

## The Simulator
This code will simulate our return based on what instructions we give it.

In [6]:
# Calculate the return for a row of oddlot data given the outlook
def calculate_odd_lot_return(row, outlook='avg'):
    sell_price = row[outlook]
    if np.isnan(sell_price) or sell_price is None:
        sell_price = row['tender px']
    return (sell_price - row['trading px']) / row['trading px']
    
# Simulate trading of as many stocks as possible at a time
def simulate_no_limit(principal, outlook):
    portfolio = pd.DataFrame({'value': [principal], 
                              'cash': [principal], 
                              'date': [pd.to_datetime('1/01/2000')]})
    # First calculate all returns given outlook
    odd_lot_data['ret'] = odd_lot_data.apply(calculate_odd_lot_return, outlook=outlook, axis=1)
    # Initialize stocks we are holding to 0
    odd_lot_data['stocks_held'] = np.zeros(len(odd_lot_data.index))
    
    trade_dates = odd_lot_data['date'].append(odd_lot_data['pay date']).sort_values().drop_duplicates()
    # Iterrate through every filing and pay day for the odd lot offers
    for trade_date in trade_dates:
        # Calculate the new portfolio value and cash after the day
        new_port_value = portfolio.tail(1)['value'].values[0]
        cash = portfolio.tail(1)['cash'].values[0]
        """ SELLING """
        # Tender any offers for the trade_date
        for odd_lot_index in odd_lot_data[odd_lot_data['pay date'] == trade_date].index:
            # Get the data for the odd_lot_offer
            odd_lot_offer = odd_lot_data.iloc[odd_lot_index]
            
            # If we have no stocks to sell, continue to next stock
            stocks_to_sell = odd_lot_offer['stocks_held']
            if stocks_to_sell == 0:
                "Continuing"
                continue
                
            # Calculate the failed and successful stocks
            failed_stocks = int(round(stocks_to_sell * failure_rate))
            successful_stocks = stocks_to_sell - failed_stocks
            trading_price = odd_lot_offer['trading px']

            # Update overall portfolio value
#             print('new_port_value: ' + str(new_port_value))
            new_port_value += successful_stocks * odd_lot_offer['ret'] * trading_price
            new_port_value -= failed_stocks * failure_penalty_rate * trading_price
#             print('new_port_value: ' + str(new_port_value))
#             print('ret: ' + str(odd_lot_offer['ret']))
            
            # Update cash
#             print('cash: ' + str(cash))
            cash += successful_stocks * (1 + odd_lot_offer['ret']) * trading_price
            cash += failed_stocks * (1 - failure_penalty_rate) * trading_price
#             print('cash: ' + str(cash))
            
            # Update stocks held
#             print(odd_lot_data.iloc[odd_lot_index]['stocks_held'])
            odd_lot_data.at[odd_lot_index, 'stocks_held'] = 0
#             print(odd_lot_data.iloc[odd_lot_index]['stocks_held'])

        
        """ BUYING """
        # Now let's look at any odd lots filed on the trade date and get the one with the highest expected return
        indexes_of_odd_lots_filed = odd_lot_data.index[odd_lot_data['date'] == trade_date]
        best_return = 0
        potential_buy_trade_ind = None
        for ind in indexes_of_odd_lots_filed:
            if odd_lot_data.iloc[ind]['ret'] > best_return:
                potential_buy_trade_ind = ind
                best_return = odd_lot_data.iloc[ind]['ret']
            
        # If there is no buy trade with a greater expected return than 0, then don't invest
        if potential_buy_trade_ind is None:
            portfolio = portfolio.append({'value': new_port_value,
                                          'cash': cash,
                                          'date': trade_date}, 
                                         ignore_index=True,)
            continue
        potential_buy_trade = odd_lot_data.iloc[potential_buy_trade_ind]
        stocks_to_buy = 0
        
        # Let's immediately buy this stock if we don't own any other stocks and it's expected return is > 0
        indexes_of_stocks_held = odd_lot_data.index[odd_lot_data['stocks_held'] > 0]
        if not len(indexes_of_stocks_held) > 0 and potential_buy_trade['ret'] > 0:
            stocks_to_buy = min(potential_buy_trade['our_share_cap'], 
                                np.floor(cash / (potential_buy_trade['trading px'] + 
                                                 potential_buy_trade['fee_cost_per_share'])))
        # Else if its expected return is greater than 0, first try to sell less valuable stocks, then buy this stock
        elif potential_buy_trade['ret'] > 0:  
            # Next, let's sell any less valuable stocks 
            indexes_of_stocks_to_sell = odd_lot_data.index[(odd_lot_data['ret'] < potential_buy_trade['ret']) & \
                                                          (odd_lot_data['stocks_held'] > 0)]
            for odd_lot_index in indexes_of_stocks_to_sell:
                # If the stock held is less valuable, sell it and buy the more valuable stock
                if odd_lot_data.iloc[odd_lot_index]['ret'] < (potential_buy_trade['ret'] - 
                                                              potential_buy_trade['fee_cost_per_share']):
                    # TODO: Consider selling at low, since that may be a more reasonable assumption than
                    # selling at the initial trading px
                    cash += odd_lot_data.iloc[odd_lot_index]['trading px'] * odd_lot_data.iloc[odd_lot_index]['stocks_held']
                    odd_lot_data.at[odd_lot_index, 'stocks_held'] = 0
            
            # Finally, let's buy as many of the new stock as we can
            stocks_to_buy = min(potential_buy_trade['our_share_cap'], 
                                np.floor(cash / (potential_buy_trade['trading px'] + 
                                                 potential_buy_trade['fee_cost_per_share'])))
        # Actually buy the stocks
        odd_lot_data.at[potential_buy_trade_ind, 'stocks_held'] = stocks_to_buy
        cash -= stocks_to_buy * potential_buy_trade['trading px']
        # Decrement the fee cost
        cash -= stocks_to_buy * potential_buy_trade['fee_cost_per_share']
        new_port_value -= stocks_to_buy * potential_buy_trade['fee_cost_per_share']
                
        portfolio = portfolio.append({'value': new_port_value,
                                      'cash': cash,
                                      'date': trade_date}, 
                                     ignore_index=True,)
        print(portfolio)
    return portfolio


# Simulate trading of 99 stocks at a time (99 stocks is the max for one account in odd lot)
def simulate_one_account_limit(principal, outlook, strategy):
    portfolio = pd.DataFrame({'value': [principal], 
                              'cash_invested': [0], 
                              'date': [pd.to_datetime('1/01/2000')]})
    # First calculate all returns given outlook
    odd_lot_data['ret'] = odd_lot_data.apply(calculate_odd_lot_return, outlook=outlook, axis=1)
    
    trade_dates = odd_lot_data['date'].append(odd_lot_data['pay date']).sort_values().drop_duplicates()
    # Iterrate through every filing and pay day for the odd lot offers
    for trade_date in trade_dates:
#         print("\n\n\n\n" + str(trade_date))
        # Get any trades for that day
        buy_trades = odd_lot_data[(odd_lot_data['date'] == trade_date) & (odd_lot_data[strategy] == 'y')]
        sell_trades = odd_lot_data[(odd_lot_data['pay date'] == trade_date) & (odd_lot_data[strategy] == 'y')]
#         print(buy_trades)
#         print(sell_trades)
        
        # Calculate the new portfolio value and cash after the day
        new_port_value = portfolio.tail(1)['value'].values[0]
        cash_invested = portfolio.tail(1)['cash_invested'].values[0]
        for i, buy_trade in buy_trades.iterrows():
            # In a buy trade, you buy 99 stocks
            cash_invested += 99 * buy_trade['trading px']
        for i, sell_trade in sell_trades.iterrows():
            # Set sell price to whatever outlook we are expecting, if the value is nan, use the tender px
            sell_price = sell_trade[outlook]
            if np.isnan(sell_price) or sell_price is None:
                sell_price = sell_trade['tender px']
            
            if sell_price > sell_trade['trading px']:
                # Sell back % of stocks at market price, i.e. the failure rate
                failed_stocks = int(round(99 * failure_rate))
                successful_stocks = 99 - failed_stocks

                # Update portfolio value after full trade
#                 new_port_value += successful_stocks * sell_price
#                 new_port_value += discounted_stocks * sell_trade['trading px']
                change_in_port_val = (successful_stocks * sell_trade['ret'] * sell_trade['trading px']) \
                                        - failed_stocks * failure_penalty_rate * sell_trade['trading px']
                new_port_value += change_in_port_val
            else:
                # Update portfolio value after full trade
                new_port_value += 99 * sell_trade['ret'] * sell_trade['trading px']
                
            # Sell back the stocks
            cash_invested -= 99 * sell_trade['trading px']
        
        portfolio = portfolio.append({'value': new_port_value,
                                      'cash_invested': cash_invested,
                                      'date': trade_date}, 
                                     ignore_index=True)
    return portfolio


def calculate_returns(portfolio, value_col):
    returns = np.empty(len(portfolio.index))
    for i in range(len(portfolio.index)):
        if i > 0:
            returns[i] = (portfolio.iloc[i][value_col] - portfolio.iloc[i-1][value_col])
        else:
            returns[i] = 0
    return returns

## Worst Case Simulation
Using outlook equals 'low', what is our return for the two strategies. Only do this if we are looking at one account.

In [7]:
portfolios = {}

if one_account_limit:
    """ One Account """
    # Calculate worst case for the naive strategy
    low_naive_portfolio = simulate_one_account_limit(10000, 'low', 'naive')
    # Calculate returns
    low_naive_portfolio['return'] = calculate_returns(low_naive_portfolio, 'value')
    low_naive_portfolio['norm return'] = low_naive_portfolio['return'] / low_naive_portfolio.iloc[0]['value']
#     print(low_naive_portfolio)

    # Calculate worst case for the conservative strategy
    low_conservative_portfolio = simulate_one_account_limit(10000, 'low', 'conservative')
    # Calculate returns
    low_conservative_portfolio['return'] = calculate_returns(low_conservative_portfolio, 'value')
    low_conservative_portfolio['norm return'] = low_conservative_portfolio['return'] / low_conservative_portfolio.iloc[0]['value']
#     print(low_conservative_portfolio)

    # Add the two worst cases to the portfolios array
    portfolios['lnp'] = low_naive_portfolio
    portfolios['lcp'] = low_conservative_portfolio

## Average Case Simulation
Using outlook == 'avg', what is our return for the two strategies

In [8]:
if one_account_limit:
    """ One Account """
    # Calculate worst case for the naive strategy
    avg_naive_portfolio = simulate_one_account_limit(10000, 'avg', 'naive')
    # Calculate returns
    avg_naive_portfolio['return'] = calculate_returns(avg_naive_portfolio, 'value')
    avg_naive_portfolio['norm return'] = avg_naive_portfolio['return'] / avg_naive_portfolio.iloc[0]['value']
    # print(avg_naive_portfolio)

    # Calculate worst case for the conservative strategy
    avg_conservative_portfolio = simulate_one_account_limit(10000, 'avg', 'conservative')
    # Calculate returns
    avg_conservative_portfolio['return'] = calculate_returns(avg_conservative_portfolio, 'value')
    avg_conservative_portfolio['norm return'] = avg_conservative_portfolio['return'] / avg_conservative_portfolio.iloc[0]['value']
    # print(avg_conservative_portfolio)

    # Add the two best cases to the portfolios array
    portfolios['anp'] = avg_naive_portfolio
    portfolios['acp'] = avg_conservative_portfolio
else:
    """ Multiple Account """
    # Calculate worst case for the naive strategy
    multi_account_portfolio = simulate_no_limit(100000, 'avg')
    # Calculate returns
    multi_account_portfolio['return'] = calculate_returns(multi_account_portfolio, 'value')
    multi_account_portfolio['norm return'] = multi_account_portfolio['return'] / multi_account_portfolio.iloc[0]['value']
    portfolios['multi'] = multi_account_portfolio
    # print(avg_naive_portfolio)

       value       cash       date
0  100000.00  100000.00 2000-01-01
1   90909.25       1.75 2000-01-26
       value       cash       date
0  100000.00  100000.00 2000-01-01
1   90909.25       1.75 2000-01-26
2   90909.25       1.75 2000-02-04
       value       cash       date
0  100000.00  100000.00 2000-01-01
1   90909.25       1.75 2000-01-26
2   90909.25       1.75 2000-02-04
3   90909.25       1.75 2000-02-24
       value       cash       date
0  100000.00  100000.00 2000-01-01
1   90909.25       1.75 2000-01-26
2   90909.25       1.75 2000-02-04
3   90909.25       1.75 2000-02-24
4   90909.25       1.75 2000-02-28
       value       cash       date
0  100000.00  100000.00 2000-01-01
1   90909.25       1.75 2000-01-26
2   90909.25       1.75 2000-02-04
3   90909.25       1.75 2000-02-24
4   90909.25       1.75 2000-02-28
5   90909.25       1.75 2000-02-29
6   90909.25       1.75 2000-03-01
       value       cash       date
0  100000.00  100000.00 2000-01-01
1   90909.25       1

40  234339.32005       3.32005 2000-10-31
           value          cash       date
0   100000.00000  100000.00000 2000-01-01
1    90909.25000       1.75000 2000-01-26
2    90909.25000       1.75000 2000-02-04
3    90909.25000       1.75000 2000-02-24
4    90909.25000       1.75000 2000-02-28
5    90909.25000       1.75000 2000-02-29
6    90909.25000       1.75000 2000-03-01
7    98814.25000   98814.25000 2000-03-02
8    98814.25000   98814.25000 2000-03-09
9    89831.50000       4.00000 2000-03-21
10   89831.50000       4.00000 2000-03-30
11   89831.50000       4.00000 2000-04-02
12   89831.50000       4.00000 2000-04-04
13   89831.50000       4.00000 2000-04-17
14   89831.50000       4.00000 2000-04-23
15   89831.50000       4.00000 2000-04-24
16  149716.50000  149716.50000 2000-04-25
17  136106.35625       4.91875 2000-05-01
18  136106.35625       4.91875 2000-05-21
19  136106.35625       4.91875 2000-05-25
20  136106.35625       4.91875 2000-06-05
21  154172.91875  154172.91875 200

           value          cash       date
0   1.000000e+05  100000.00000 2000-01-01
1   9.090925e+04       1.75000 2000-01-26
2   9.090925e+04       1.75000 2000-02-04
3   9.090925e+04       1.75000 2000-02-24
4   9.090925e+04       1.75000 2000-02-28
5   9.090925e+04       1.75000 2000-02-29
6   9.090925e+04       1.75000 2000-03-01
7   9.881425e+04   98814.25000 2000-03-02
8   9.881425e+04   98814.25000 2000-03-09
9   8.983150e+04       4.00000 2000-03-21
10  8.983150e+04       4.00000 2000-03-30
11  8.983150e+04       4.00000 2000-04-02
12  8.983150e+04       4.00000 2000-04-04
13  8.983150e+04       4.00000 2000-04-17
14  8.983150e+04       4.00000 2000-04-23
15  8.983150e+04       4.00000 2000-04-24
16  1.497165e+05  149716.50000 2000-04-25
17  1.361064e+05       4.91875 2000-05-01
18  1.361064e+05       4.91875 2000-05-21
19  1.361064e+05       4.91875 2000-05-25
20  1.361064e+05       4.91875 2000-06-05
21  1.541729e+05  154172.91875 2000-06-06
22  1.541729e+05  154172.91875 200

[122 rows x 3 columns]
            value          cash       date
0    1.000000e+05  1.000000e+05 2000-01-01
1    9.090925e+04  1.750000e+00 2000-01-26
2    9.090925e+04  1.750000e+00 2000-02-04
3    9.090925e+04  1.750000e+00 2000-02-24
4    9.090925e+04  1.750000e+00 2000-02-28
5    9.090925e+04  1.750000e+00 2000-02-29
6    9.090925e+04  1.750000e+00 2000-03-01
7    9.881425e+04  9.881425e+04 2000-03-02
8    9.881425e+04  9.881425e+04 2000-03-09
9    8.983150e+04  4.000000e+00 2000-03-21
10   8.983150e+04  4.000000e+00 2000-03-30
11   8.983150e+04  4.000000e+00 2000-04-02
12   8.983150e+04  4.000000e+00 2000-04-04
13   8.983150e+04  4.000000e+00 2000-04-17
14   8.983150e+04  4.000000e+00 2000-04-23
15   8.983150e+04  4.000000e+00 2000-04-24
16   1.497165e+05  1.497165e+05 2000-04-25
17   1.361064e+05  4.918750e+00 2000-05-01
18   1.361064e+05  4.918750e+00 2000-05-21
19   1.361064e+05  4.918750e+00 2000-05-25
20   1.361064e+05  4.918750e+00 2000-06-05
21   1.541729e+05  1.541729e+05

            value          cash       date
0    1.000000e+05  1.000000e+05 2000-01-01
1    9.090925e+04  1.750000e+00 2000-01-26
2    9.090925e+04  1.750000e+00 2000-02-04
3    9.090925e+04  1.750000e+00 2000-02-24
4    9.090925e+04  1.750000e+00 2000-02-28
5    9.090925e+04  1.750000e+00 2000-02-29
6    9.090925e+04  1.750000e+00 2000-03-01
7    9.881425e+04  9.881425e+04 2000-03-02
8    9.881425e+04  9.881425e+04 2000-03-09
9    8.983150e+04  4.000000e+00 2000-03-21
10   8.983150e+04  4.000000e+00 2000-03-30
11   8.983150e+04  4.000000e+00 2000-04-02
12   8.983150e+04  4.000000e+00 2000-04-04
13   8.983150e+04  4.000000e+00 2000-04-17
14   8.983150e+04  4.000000e+00 2000-04-23
15   8.983150e+04  4.000000e+00 2000-04-24
16   1.497165e+05  1.497165e+05 2000-04-25
17   1.361064e+05  4.918750e+00 2000-05-01
18   1.361064e+05  4.918750e+00 2000-05-21
19   1.361064e+05  4.918750e+00 2000-05-25
20   1.361064e+05  4.918750e+00 2000-06-05
21   1.541729e+05  1.541729e+05 2000-06-06
22   1.5417

...

            value          cash       date
0    1.000000e+05  1.000000e+05 2000-01-01
1    9.090925e+04  1.750000e+00 2000-01-26
2    9.090925e+04  1.750000e+00 2000-02-04
3    9.090925e+04  1.750000e+00 2000-02-24
4    9.090925e+04  1.750000e+00 2000-02-28
5    9.090925e+04  1.750000e+00 2000-02-29
6    9.090925e+04  1.750000e+00 2000-03-01
7    9.881425e+04  9.881425e+04 2000-03-02
8    9.881425e+04  9.881425e+04 2000-03-09
9    8.983150e+04  4.000000e+00 2000-03-21
10   8.983150e+04  4.000000e+00 2000-03-30
11   8.983150e+04  4.000000e+00 2000-04-02
12   8.983150e+04  4.000000e+00 2000-04-04
13   8.983150e+04  4.000000e+00 2000-04-17
14   8.983150e+04  4.000000e+00 2000-04-23
15   8.983150e+04  4.000000e+00 2000-04-24
16   1.497165e+05  1.497165e+05 2000-04-25
17   1.361064e+05  4.918750e+00 2000-05-01
18   1.361064e+05  4.918750e+00 2000-05-21
19   1.361064e+05  4.918750e+00 2000-05-25
20   1.361064e+05  4.918750e+00 2000-06-05
21   1.541729e+05  1.541729e+05 2000-06-06
22   1.5417

[555 rows x 3 columns]
            value          cash       date
0    1.000000e+05  1.000000e+05 2000-01-01
1    9.090925e+04  1.750000e+00 2000-01-26
2    9.090925e+04  1.750000e+00 2000-02-04
3    9.090925e+04  1.750000e+00 2000-02-24
4    9.090925e+04  1.750000e+00 2000-02-28
5    9.090925e+04  1.750000e+00 2000-02-29
6    9.090925e+04  1.750000e+00 2000-03-01
7    9.881425e+04  9.881425e+04 2000-03-02
8    9.881425e+04  9.881425e+04 2000-03-09
9    8.983150e+04  4.000000e+00 2000-03-21
10   8.983150e+04  4.000000e+00 2000-03-30
11   8.983150e+04  4.000000e+00 2000-04-02
12   8.983150e+04  4.000000e+00 2000-04-04
13   8.983150e+04  4.000000e+00 2000-04-17
14   8.983150e+04  4.000000e+00 2000-04-23
15   8.983150e+04  4.000000e+00 2000-04-24
16   1.497165e+05  1.497165e+05 2000-04-25
17   1.361064e+05  4.918750e+00 2000-05-01
18   1.361064e+05  4.918750e+00 2000-05-21
19   1.361064e+05  4.918750e+00 2000-05-25
20   1.361064e+05  4.918750e+00 2000-06-05
21   1.541729e+05  1.541729e+05

            value          cash       date
0    1.000000e+05  1.000000e+05 2000-01-01
1    9.090925e+04  1.750000e+00 2000-01-26
2    9.090925e+04  1.750000e+00 2000-02-04
3    9.090925e+04  1.750000e+00 2000-02-24
4    9.090925e+04  1.750000e+00 2000-02-28
5    9.090925e+04  1.750000e+00 2000-02-29
6    9.090925e+04  1.750000e+00 2000-03-01
7    9.881425e+04  9.881425e+04 2000-03-02
8    9.881425e+04  9.881425e+04 2000-03-09
9    8.983150e+04  4.000000e+00 2000-03-21
10   8.983150e+04  4.000000e+00 2000-03-30
11   8.983150e+04  4.000000e+00 2000-04-02
12   8.983150e+04  4.000000e+00 2000-04-04
13   8.983150e+04  4.000000e+00 2000-04-17
14   8.983150e+04  4.000000e+00 2000-04-23
15   8.983150e+04  4.000000e+00 2000-04-24
16   1.497165e+05  1.497165e+05 2000-04-25
17   1.361064e+05  4.918750e+00 2000-05-01
18   1.361064e+05  4.918750e+00 2000-05-21
19   1.361064e+05  4.918750e+00 2000-05-25
20   1.361064e+05  4.918750e+00 2000-06-05
21   1.541729e+05  1.541729e+05 2000-06-06
22   1.5417

            value          cash       date
0    1.000000e+05  1.000000e+05 2000-01-01
1    9.090925e+04  1.750000e+00 2000-01-26
2    9.090925e+04  1.750000e+00 2000-02-04
3    9.090925e+04  1.750000e+00 2000-02-24
4    9.090925e+04  1.750000e+00 2000-02-28
5    9.090925e+04  1.750000e+00 2000-02-29
6    9.090925e+04  1.750000e+00 2000-03-01
7    9.881425e+04  9.881425e+04 2000-03-02
8    9.881425e+04  9.881425e+04 2000-03-09
9    8.983150e+04  4.000000e+00 2000-03-21
10   8.983150e+04  4.000000e+00 2000-03-30
11   8.983150e+04  4.000000e+00 2000-04-02
12   8.983150e+04  4.000000e+00 2000-04-04
13   8.983150e+04  4.000000e+00 2000-04-17
14   8.983150e+04  4.000000e+00 2000-04-23
15   8.983150e+04  4.000000e+00 2000-04-24
16   1.497165e+05  1.497165e+05 2000-04-25
17   1.361064e+05  4.918750e+00 2000-05-01
18   1.361064e+05  4.918750e+00 2000-05-21
19   1.361064e+05  4.918750e+00 2000-05-25
20   1.361064e+05  4.918750e+00 2000-06-05
21   1.541729e+05  1.541729e+05 2000-06-06
22   1.5417

[780 rows x 3 columns]
            value          cash       date
0    1.000000e+05  1.000000e+05 2000-01-01
1    9.090925e+04  1.750000e+00 2000-01-26
2    9.090925e+04  1.750000e+00 2000-02-04
3    9.090925e+04  1.750000e+00 2000-02-24
4    9.090925e+04  1.750000e+00 2000-02-28
5    9.090925e+04  1.750000e+00 2000-02-29
6    9.090925e+04  1.750000e+00 2000-03-01
7    9.881425e+04  9.881425e+04 2000-03-02
8    9.881425e+04  9.881425e+04 2000-03-09
9    8.983150e+04  4.000000e+00 2000-03-21
10   8.983150e+04  4.000000e+00 2000-03-30
11   8.983150e+04  4.000000e+00 2000-04-02
12   8.983150e+04  4.000000e+00 2000-04-04
13   8.983150e+04  4.000000e+00 2000-04-17
14   8.983150e+04  4.000000e+00 2000-04-23
15   8.983150e+04  4.000000e+00 2000-04-24
16   1.497165e+05  1.497165e+05 2000-04-25
17   1.361064e+05  4.918750e+00 2000-05-01
18   1.361064e+05  4.918750e+00 2000-05-21
19   1.361064e+05  4.918750e+00 2000-05-25
20   1.361064e+05  4.918750e+00 2000-06-05
21   1.541729e+05  1.541729e+05

## Best Case Simulation
Using outlook equals 'high', what is our return for the two strategies

In [9]:
if one_account_limit:
    """ One Account """
    # Calculate worst case for the naive strategy
    high_naive_portfolio = simulate_one_account_limit(10000, 'high', 'naive')
    # Calculate returns
    high_naive_portfolio['return'] = calculate_returns(high_naive_portfolio, 'value')
    high_naive_portfolio['norm return'] = high_naive_portfolio['return'] / high_naive_portfolio.iloc[0]['value']
    # print(high_naive_portfolio)

    # Calculate worst case for the conservative strategy
    high_conservative_portfolio = simulate_one_account_limit(10000, 'high', 'conservative')
    # Calculate returns
    high_conservative_portfolio['return'] = calculate_returns(high_conservative_portfolio, 'value')
    high_conservative_portfolio['norm return'] = high_conservative_portfolio['return'] / high_conservative_portfolio.iloc[0]['value']
    # print(high_conservative_portfolio)

    # Add the two best cases to the portfolios array
    portfolios['hnp'] = high_naive_portfolio
    portfolios['hcp'] = high_conservative_portfolio

## Fill in Dates
Fills in the dates outside of trading dates

In [10]:
# for portfolio_name in portfolios.keys():
    
#     portfolio = portfolios[portfolio_name]
    
#     # Get simulation date range to fill in the values
#     date_range = pd.date_range(start_date, end_date)
    
#     new_portfolio =
#     if one_account_limit:
#         new_portfolio = pd.DataFrame({'value': [], 'cash_invested': [], 'date': date_range})
#     else:
#         new_portfolio = pd.DataFrame({'value': [], 'cash': [], 'date': date_range})


## Absolute Returns
Get the absolute returns for the portfolios.


In [11]:
absolute_portfolios = None
for name, portfolio in portfolios.items():
    portfolio['year'] = pd.DatetimeIndex(portfolio['date']).year.astype(int)
    
    portfolio['change'] = portfolio['value'].shift(-1) - portfolio['value']

    # Calculate annualized returns
    absolute_returns = []
    years = np.sort(portfolio['year'].unique())
    for year in years:
        year_data = portfolio[portfolio['year'] == year]
        start_value = year_data.iloc[0]['value']
        end_value = year_data.tail(1).iloc[0]['value']
        ####
        # TODO: divide by start value once we are full utilizing returns!!!
        ####
        absolute_returns.append(end_value - start_value)
        
    if absolute_portfolios is None:
        absolute_portfolios = pd.DataFrame({name: absolute_returns},
                                            index=years)
    else:
        absolute_portfolios[name] = absolute_returns
                
print(absolute_portfolios)
absolute_portfolios.to_csv('./data/results-mult-account/fee_10_percent_abs_annual_port_rets.csv')

             multi
2000  2.877593e+05
2001  1.261177e+06
2002  1.590789e+06
2003  6.612163e+04
2004 -4.222608e+05
2005 -1.945089e+05
2006 -4.557604e+05
2007  4.817810e+04
2008  5.071264e+05
2009  2.082872e+05
2010 -4.337664e+05
2011  3.821976e+05
2012 -4.011876e+05
2013  2.298018e+04
2014  4.827183e+05
2015 -6.365017e+05
2016  4.314664e+04
2017  3.141258e+05
2018  3.610390e+04


## Annualizing Data
We will now group the data up by year so that we can annualize it and analyze it better, starting in 2000.

In [12]:
annualized_portfolios = None
for name, portfolio in portfolios.items():
    portfolio['year'] = pd.DatetimeIndex(portfolio['date']).year.astype(int)

    # Calculate annualized returns
    annualized_returns = []
    years = np.sort(portfolio['year'].unique())
    for year in years:
        year_data = portfolio[portfolio['year'] == year]
        start_value = year_data.iloc[0]['value']
        end_value = year_data.tail(1).iloc[0]['value']

        # If there is a one account cap for any given tender offer, 
        # we assume that you will only need $50,000 in order to max out a single account
        # for each tender offer. Thus, we divide our change in portfolio value across the
        # year by $50,000, as this would be the principal needed to safely follow this strategy
        if one_account_limit:
            annualized_returns.append((end_value - start_value) / 50000)
        else:
            annualized_returns.append((end_value - start_value) / start_value)
        
    if annualized_portfolios is None:
        annualized_portfolios = pd.DataFrame({name: annualized_returns},
                                            index=years)
    else:
        annualized_portfolios[name] = annualized_returns
                
print(annualized_portfolios)
annualized_portfolios.to_csv('./data/results-mult-account/fee_10_percent_annual_port_rets.csv')

         multi
2000  2.877593
2001  3.483390
2002  0.980014
2003  0.019995
2004 -0.125188
2005 -0.065918
2006 -0.165355
2007  0.020943
2008  0.237513
2009  0.063734
2010 -0.137253
2011  0.140175
2012 -0.129051
2013  0.007717
2014  0.176956
2015 -0.185814
2016  0.015470
2017  0.114306
2018  0.012969


## Analyzing portfolios
We will now calculate things like the standard deviation of returns, sharpe ratios, overall returns, etc.

In [13]:
analysis_data = pd.DataFrame(columns={'strategy', 'final_ret', 'ret_std', 'sharpe_ratio'})

# TODO: Use anualized returns/std deviation to calculate this stuff
for strategy in annualized_portfolios:
    portfolio = annualized_portfolios[strategy]
    
    row = {}
    row['strategy'] = strategy
    
    # Calculate cash needed to maintain a single account:
    
    # Calculate final return
    final_ret = 1
    for ret in portfolio.values:
        final_ret = final_ret * (1 + ret)
    row['final_ret'] = final_ret - 1
    
    # Calculate standard deviations
    row['ret_std'] = portfolio.std()
    
    # Calculate Sharpe Ratio using MEAN 20 YEAR TREASURY RATE as risk free rate
#     Sharpe ratio = (return - risk_free_return) / std_dev
    row['sharpe_ratio'] = (row['final_ret'] - .024) / row['ret_std']
    
    
    # Add to analysis dataframe
    analysis_data = analysis_data.append(row, ignore_index=True)

print(analysis_data)

   sharpe_ratio strategy  final_ret   ret_std
0     28.874489    multi  29.518178  1.021461


## Getting Time Series Data
Just save the portfolio data

In [14]:
# for name, portfolio in portfolios.items():
#     if one_account_limit:
#         time_series_data = portfolio.drop(['cash_invested', 'return','norm return'], axis=1).set_index('date')
#     else:
#         time_series_data = portfolio.drop(['cash', 'return','norm return'], axis=1).set_index('date')

#     time_series_data.to_csv('./data/results-mult-account/time_series_data_' + name + '.csv')

## Get Odd lots/Tender lots by month
Just like it sounds

In [33]:
# Get Odd lots by month
odd_lot_data['year'] = pd.DatetimeIndex(odd_lot_data['date']).year.astype(int)
odd_lot_data['month'] = pd.DatetimeIndex(odd_lot_data['date']).month.astype(int)
odd_lots_by_month = odd_lot_data.groupby(['year', 'month']).count()['cik'].reset_index()
odd_lots_by_month.rename({'cik': 'odd lot offers'}, inplace=True, axis=1)
odd_lots_by_month.to_csv('odd_lot_offers_by_month.csv', index=False)
print(odd_lots_by_month)


# Get tender offers by month
tender_offer_data = pd.read_csv('./data/cleaned-data/tender_offer_data.csv')
# Remove any data rows where there have been or duplicates
tender_offer_data.drop_duplicates(subset=['cik', 'date'], inplace = True)
# Make the date columns into actual python datetimes
tender_offer_data['date'] = pd.to_datetime(tender_offer_data['date'])
# Only look at data inside of date range
tender_offer_data = tender_offer_data[(tender_offer_data['date'] > start_date) & (tender_offer_data['date'] < end_date)]

tender_offer_data['year'] = pd.DatetimeIndex(tender_offer_data['date']).year.astype(int)
tender_offer_data['month'] = pd.DatetimeIndex(tender_offer_data['date']).month.astype(int)
tender_offer_data = tender_offer_data.groupby(['year', 'month']).count()['cik'].reset_index()
tender_offer_data.rename({'cik': 'tender offer'}, inplace=True, axis=1)
tender_offer_data.to_csv('tender_offers_by_month.csv', index=False)
print(tender_offer_data)

     year  month  odd lot offers
0    2000      1               2
1    2000      2               3
2    2000      3               3
3    2000      4               2
4    2000      5               1
5    2000      6               1
6    2000      7               1
7    2000      8               4
8    2000      9               5
9    2000     10               3
10   2000     11               3
11   2000     12               5
12   2001      1               1
13   2001      2               3
14   2001      3               2
15   2001      4               4
16   2001      5               2
17   2001      6               2
18   2001      7               4
19   2001      8               3
20   2001      9               1
21   2001     10               1
22   2001     11               4
23   2001     12               3
24   2002      1               1
25   2002      2               1
26   2002      3               1
27   2002      4               3
28   2002      5               4
29   2002 